In [22]:
import pandas as pd
import numpy as np

np.random.seed(42)

n_responses = 1000

age = np.random.randint(18, 70, size=n_responses)
# gender = np.random.choice([0, 1], size=n_responses) # 0 = Muž, 1 = Žena
gender = np.random.choice(['Muž', 'Žena'], size=n_responses)
# education = np.random.choice([1, 2, 3], size=n_responses)  # 1 = Primary, 2 = Secondary, 3 = Tertiary
education = np.random.choice(['Primary', 'Secondary', 'Tertiary'], size=n_responses)
# city_size = np.random.choice([1, 2, 3], size=n_responses)  # 1 = Small, 2 = Medium, 3 = Large
city_size = np.random.choice(['Small', 'Medium', 'Large'], size=n_responses)

# Odpovědi na otázky
question1 = np.random.randint(1, 6, size=n_responses)  # Škála 1-5
# question2 = np.random.choice([1, 0], size=n_responses) # 1 = Ano, 0 = Ne
question2 = np.random.choice(['Ano', 'Ne'], size=n_responses)
question3 = np.random.normal(50, 10, size=n_responses)

data = pd.DataFrame({
    'Age': age,
    'Gender': gender,
    'Education': education,
    'City_Size': city_size,
    'Question1': question1,
    'Question2': question2,
    'Question3': question3
})

data.to_csv('../data/survey_data.csv', index=False)

In [17]:
from scipy import stats

# data se musi previst na numericke hodnoty

def check_anova_assumptions(data):
    # Předpoklad: Normalita dat
    # Test Shapiro-Wilk
    _, shapiro_p = stats.shapiro(data)
    if shapiro_p > 0.05:
        print("Data mají normální rozdělení (Shapiro-Wilk test, p = {:.4f})".format(shapiro_p))
    else:
        print("Data nemají normální rozdělení (Shapiro-Wilk test, p = {:.4f})".format(shapiro_p))

    # Předpoklad: Homoskedasticita
    # Test Levene
    _, levene_p = stats.levene(data.loc[data["Gender"] == 0, "Question1"], data.loc[data["Gender"] == 1, "Question1"])
    if levene_p > 0.05:
        print("Data jsou homoskedastická (Levene test, p = {:.4f})".format(levene_p))
    else:
        print("Data nejsou homoskedastická (Levene test, p = {:.4f})".format(levene_p))

check_anova_assumptions(data)

Data nemají normální rozdělení (Shapiro-Wilk test, p = 0.0000)
Data jsou homoskedastická (Levene test, p = 0.2715)


/home/petka/PROJECTS/KMA-SZZ/.venv/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning:

scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 7000.



In [25]:
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px

# Načtení dat
data = pd.read_csv('../data/survey_data.csv')

# 2) Explorační analýza dat
# Prozkoumání vztahů mezi proměnnými a demografickými faktory pomocí Seaborn

# Histogram věku
# fig = px.histogram(data, x='Age', nbins=20, title='Age Distribution', labels={'Age': 'Age', 'count': 'Frequency'})
# fig.show()
bar_data = data['Age'].value_counts().reset_index()
fig = px.bar(bar_data, x='Age', y='count', title='Age Distribution', labels={'index': 'Age', 'Age': 'Frequency'})
fig.show()

# Boxplot věku podle pohlaví
fig = px.box(data, x='Gender', y='Age', title='Age Distribution by Gender', labels={'Gender': 'Gender', 'Age': 'Age'})
fig.show()

# Scatterplot mezi věkem a odpovědí na otázku 3
fig = px.scatter(data, x='Age', y='Question3', title='Scatterplot: Age vs Question 3', labels={'Age': 'Age', 'Question3': 'Question 3'})
fig.show()

# 3) Formulace statistických hypotéz
# Hypotéza: Existuje rozdíl ve skóre otázky 1 mezi různými vzděláními.

# 4) Testování statistických hypotéz
# Analýza rozptylu (ANOVA)
anova_result = stats.f_oneway(data[data['Education'] == 'Primary']['Question1'],
                               data[data['Education'] == 'Secondary']['Question1'],
                               data[data['Education'] == 'Tertiary']['Question1'])
print("ANOVA results for Question 1 by Education:")
print(anova_result)

# Neparametrický Kruskal-Wallis test pro srovnání vzdělání a otázky 1
kruskal_result = stats.kruskal(data[data['Education'] == 'Primary']['Question1'],
                                data[data['Education'] == 'Secondary']['Question1'],
                                data[data['Education'] == 'Tertiary']['Question1'])
print("Kruskal-Wallis results for Question 1 by Education:")
print(kruskal_result)

# Interval spolehlivosti pro průměr věku
age_mean_ci = stats.t.interval(0.95, len(data['Age']) - 1, loc=np.mean(data['Age']), scale=stats.sem(data['Age']))
print("Confidence Interval for Age Mean:")
print(age_mean_ci)

# 5) Komentovaný report
# V tomto bodě bychom vytvořili report obsahující výsledky a interpretace z kroků 2, 4 a 5.



ANOVA results for Question 1 by Education:
F_onewayResult(statistic=3.125898171114664, pvalue=0.04432801944538322)
Kruskal-Wallis results for Question 1 by Education:
KruskalResult(statistic=6.213455362600374, pvalue=0.04474714293621707)
Confidence Interval for Age Mean:
(42.8887363676189, 44.74926363238111)
